In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
import numpy as np
import random
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, collect_set, mean
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, StringIndexer,VectorAssembler,OneHotEncoder,StandardScaler

In [3]:
scala_version = '2.12'
spark_version = '3.5.0'
packages = [ f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}' ,
                                                'org.apache.kafka:kafka-clients:3.6.0']
spark = SparkSession.builder.master("local")\
                            .appName("Bigmart Regresion System")\
                            .config("spark.jars.packages", ",".join(packages))\
                            .getOrCreate()

23/11/26 16:41:22 WARN Utils: Your hostname, dothinh.local resolves to a loopback address: 127.0.0.1; using 192.168.1.2 instead (on interface en0)
23/11/26 16:41:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/usr/local/Cellar/apache-spark/3.5.0/libexec/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/dothinhtpr247gmai.com/.ivy2/cache
The jars for the packages stored in: /Users/dothinhtpr247gmai.com/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f51b5b92-81c1-4476-bc75-9eb37d4fcdcf;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.0 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
	found org.apache.kafka#kafka-clients;3.6.0 in central
	found com.github.

In [4]:
train_df = spark.read\
				.options(header = True, delimiter = ',')\
				.csv("data/Train.csv")

In [5]:
train_df.printSchema()

root
 |-- Item_Identifier: string (nullable = true)
 |-- Item_Weight: string (nullable = true)
 |-- Item_Fat_Content: string (nullable = true)
 |-- Item_Visibility: string (nullable = true)
 |-- Item_Type: string (nullable = true)
 |-- Item_MRP: string (nullable = true)
 |-- Outlet_Identifier: string (nullable = true)
 |-- Outlet_Establishment_Year: string (nullable = true)
 |-- Outlet_Size: string (nullable = true)
 |-- Outlet_Location_Type: string (nullable = true)
 |-- Outlet_Type: string (nullable = true)
 |-- Item_Outlet_Sales: string (nullable = true)



In [6]:
train_df = train_df.withColumn('Item_Weight', col('Item_Weight').cast('float'))
train_df = train_df.withColumn('Item_Visibility', col('Item_Visibility').cast('float'))
train_df = train_df.withColumn('Item_MRP', col('Item_MRP').cast('float'))
train_df = train_df.withColumn('Item_Outlet_Sales', col('Item_Outlet_Sales').cast('float'))
train_df = train_df.withColumn('Outlet_Establishment_Year', col('Outlet_Establishment_Year').cast('int'))

In [7]:
train_df.toPandas().head(10)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300000,Low Fat,0.016047,Dairy,249.809204,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.137939
1,DRC01,5.920000,Regular,0.019278,Soft Drinks,48.269199,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.422791
2,FDN15,17.500000,Low Fat,0.016760,Meat,141.617996,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.270020
3,FDX07,19.200001,Regular,0.000000,Fruits and Vegetables,182.095001,OUT010,1998,None,Tier 3,Grocery Store,732.380005
4,NCD19,8.930000,Low Fat,0.000000,Household,53.861401,OUT013,1987,High,Tier 3,Supermarket Type1,994.705200
5,FDP36,10.395000,Regular,0.000000,Baking Goods,51.400799,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.608826
6,FDO10,13.650000,Regular,0.012741,Snack Foods,57.658798,OUT013,1987,High,Tier 3,Supermarket Type1,343.552795
7,FDP10,NaN,Low Fat,0.127470,Snack Foods,107.762199,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.763672
8,FDH17,16.200001,Regular,0.016687,Frozen Foods,96.972603,OUT045,2002,None,Tier 2,Supermarket Type1,1076.598633
9,FDU28,19.200001,Regular,0.094450,Frozen Foods,187.821396,OUT017,2007,None,Tier 2,Supermarket Type1,4710.535156


In [8]:
train_df.filter(train_df.Outlet_Type == "Supermarket Type1").toPandas().head(10)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300000,Low Fat,0.016047,Dairy,249.809204,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.137939
1,FDN15,17.500000,Low Fat,0.016760,Meat,141.617996,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.270020
2,NCD19,8.930000,Low Fat,0.000000,Household,53.861401,OUT013,1987,High,Tier 3,Supermarket Type1,994.705200
3,FDO10,13.650000,Regular,0.012741,Snack Foods,57.658798,OUT013,1987,High,Tier 3,Supermarket Type1,343.552795
4,FDH17,16.200001,Regular,0.016687,Frozen Foods,96.972603,OUT045,2002,None,Tier 2,Supermarket Type1,1076.598633
5,FDU28,19.200001,Regular,0.094450,Frozen Foods,187.821396,OUT017,2007,None,Tier 2,Supermarket Type1,4710.535156
6,FDY07,11.800000,Low Fat,0.000000,Fruits and Vegetables,45.540199,OUT049,1999,Medium,Tier 1,Supermarket Type1,1516.026611
7,FDA03,18.500000,Regular,0.045464,Dairy,144.110199,OUT046,1997,Small,Tier 1,Supermarket Type1,2187.153076
8,FDX32,15.100000,Regular,0.100014,Fruits and Vegetables,145.478607,OUT049,1999,Medium,Tier 1,Supermarket Type1,1589.264648
9,FDS46,17.600000,Regular,0.047257,Snack Foods,119.678200,OUT046,1997,Small,Tier 1,Supermarket Type1,2145.207520


In [9]:
train_df.select(['Outlet_Type','Outlet_Size','Outlet_Location_Type']).groupBy('Outlet_Type').pivot('Outlet_Location_Type').agg(collect_set('Outlet_Size')).toPandas()

,Outlet_Type,Tier 1,Tier 2,Tier 3
0,Supermarket Type3,[],[],[Medium]
1,Grocery Store,[Small],[],[]
2,Supermarket Type2,[],[],[Medium]
3,Supermarket Type1,"[Medium, Small]",[Small],[High]


In [10]:
train_df.select(['Outlet_Type','Outlet_Size']).groupBy('Outlet_Type').agg(collect_set('Outlet_Size')).toPandas()

,Outlet_Type,collect_set(Outlet_Size)
0,Supermarket Type3,[Medium]
1,Grocery Store,[Small]
2,Supermarket Type2,[Medium]
3,Supermarket Type1,"[Medium, Small, High]"


In [11]:
train_df = train_df.withColumn("Outlet_Size", when(train_df.Outlet_Size.isNull(),
                                                        when(train_df.Outlet_Size.isin('Supermarket Type3',
                                                                                       'Supermarket Type2'),
                                                            'Medium')\
                                                        .when(train_df.Outlet_Size == 'Grocery Store','Small')\
                                                        .when(train_df.Outlet_Location_Type == 'Tier 3','High')\
                                                        .when(train_df.Outlet_Location_Type == 'Tier 2','Small')\
                                                        .when(train_df.Outlet_Location_Type == 'Tier 1',random.choice(['Medium', 'Small']))\
                                                    )\
                                                    .otherwise(train_df.Outlet_Size))

In [12]:
# filling the missing values in "Item_weight column" with "Mean" value
mean_item_weight = train_df.agg(mean('Item_Weight')).collect()[0][0]
train_df = train_df.na.fill( float(mean_item_weight) ,['Item_Weight'])

In [13]:
mean_item_weight

12.857645339263398

In [14]:
train_df = train_df.replace(to_replace ={'low fat':'Low Fat','LF':'Low Fat', 'reg':'Regular'}, subset = ['Item_Fat_Content'])

In [15]:
# train_column = list(train_df.columns)[:-1]
stringTypeColumn = ['Item_Identifier','Item_Fat_Content',
				'Item_Type','Outlet_Identifier',
				'Outlet_Size','Outlet_Location_Type','Outlet_Type']

numberTypeColumn = ['Item_Weight','Item_Visibility','Item_MRP','Outlet_Establishment_Year']

actualColumn = 'Item_Outlet_Sales'

In [16]:
train_pd = train_df.toPandas()
for c in train_pd:
    print(c, len(train_pd[c].unique()))

Item_Identifier 1559
Item_Weight 416
Item_Fat_Content 2
Item_Visibility 7879
Item_Type 16
Item_MRP 5938
Outlet_Identifier 10
Outlet_Establishment_Year 9
Outlet_Size 3
Outlet_Location_Type 3
Outlet_Type 4
Item_Outlet_Sales 3493


## Model

In [17]:
class RegressionModel():
    df = None
    categoricalCols = None
    continuousCols = None
    labelCol = None
    model = None
    def __init__(self,df,categoricalCols,continuousCols,labelCol):
        self.df = df
        self.continuousCols = continuousCols
        self.categoricalCols = categoricalCols
        self.labelCol = labelCol

    def fitModel(self):
        
        
        indexers = StringIndexer(inputCols=self.categoricalCols, 
                                 outputCols=[f"{c}_indexed" for c in self.categoricalCols],
                                 handleInvalid="keep")

        # default setting: dropLast=True
        encoders = OneHotEncoder(inputCols = indexers.getOutputCols(),
                                 outputCols=[ f"{c}_encoded" for c in indexers.getOutputCols()])

        assembler = VectorAssembler(inputCols = encoders.getInputCols() + self.continuousCols , outputCol = "features")
        rf = RandomForestRegressor(featuresCol=assembler.getOutputCol(),
                                   labelCol=self.labelCol,
                                   maxBins=1560, numTrees=10, maxDepth=20)
        
        # normalizer = StandardScaler(inputCol = "features", outputCol="normalized_features")
        # normalizer_X = StandardScaler(inputCol = self.labelCol, outputCol="normalized_label")

        pipeline = Pipeline(stages = [indexers, encoders, assembler, rf])

        self.model = pipeline.fit(self.df)

    def transform(self, df):
        
        data = self.model.transform(df)
        return data.select('features', col(self.labelCol).alias('label'),'prediction')
    
    def savePipline(self):
        self.model.save('REPROCESSING_PIPLINE_MODEL1/')


In [18]:
from pyspark.ml.evaluation import RegressionEvaluator
def evaluate(prediction):
    # Select (prediction, true label) and compute test error
    evaluator = RegressionEvaluator(labelCol="label",
                                    predictionCol="prediction",
                                    metricName="rmse")

    rmse = evaluator.evaluate(prediction)
    print("Root Mean Squared Error (RMSE) on test data = %0.2f" % float(rmse))

In [19]:
resModel = RegressionModel(train_df, stringTypeColumn, numberTypeColumn, actualColumn)
resModel.fitModel()

23/11/26 16:41:39 WARN DAGScheduler: Broadcasting large task binary with size 1040.4 KiB
23/11/26 16:41:41 WARN DAGScheduler: Broadcasting large task binary with size 1385.2 KiB
23/11/26 16:41:45 WARN DAGScheduler: Broadcasting large task binary with size 1861.3 KiB
23/11/26 16:41:52 WARN DAGScheduler: Broadcasting large task binary with size 2.5 MiB
23/11/26 16:42:01 WARN DAGScheduler: Broadcasting large task binary with size 3.4 MiB
23/11/26 16:42:16 WARN DAGScheduler: Broadcasting large task binary with size 4.5 MiB
23/11/26 16:42:36 WARN DAGScheduler: Broadcasting large task binary with size 5.9 MiB
23/11/26 16:42:58 WARN DAGScheduler: Broadcasting large task binary with size 7.5 MiB
23/11/26 16:43:22 WARN DAGScheduler: Broadcasting large task binary with size 9.1 MiB
23/11/26 16:43:47 WARN DAGScheduler: Broadcasting large task binary with size 10.6 MiB
23/11/26 16:44:10 WARN DAGScheduler: Broadcasting large task binary with size 11.9 MiB
23/11/26 16:44:35 WARN DAGScheduler: Broadc

In [20]:
predict = resModel.transform(train_df)

In [21]:
evaluate(predict)

Root Mean Squared Error (RMSE) on test data = 691.35


In [22]:
resModel.savePipline()

23/11/26 16:45:16 WARN TaskSetManager: Stage 83 contains a task of very large size (10150 KiB). The maximum recommended task size is 1000 KiB.
